In [9]:
# -*- coding:utf-8 -*-
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import time, os
import api_key as api
import common_tools as common_tools

api_key = api.openai_api_key()
# 将API Key保存为环境变量
os.environ["OPENAI_API_KEY"] = api_key
model_name = "gpt-4o"

model = ChatOpenAI(
    model_name=model_name,
    streaming=True
)

output_parser = StrOutputParser()

def proofreading():
    prompt_content = common_tools.read_file("/Users/Daglas/dalong.llm/dalong.langchain/prompt_proofreading.md")
    origin_content = common_tools.read_file("/Users/Daglas/Desktop/input.md")
    out_filename = "/Users/Daglas/Desktop/output.md"

    prompt = ChatPromptTemplate.from_messages([
        ("system", prompt_content),
        ("user", "{text}")
    ])
    
    suggestion_chain = prompt | model | output_parser
    
    suggestion = suggestion_chain.invoke({
      "text":origin_content
    })

    suggest_and_pass = RunnableParallel(
      {"suggestion": suggestion_chain, "text": RunnablePassthrough()}
    )

    chain = suggest_and_pass | prompt | model | output_parser
#     chain = suggest_and_pass | finish_prompt | model | output_parser

    response = chain.invoke({
      "text":origin_content,
      "suggestion":suggestion
    })

#     out_content = common_tools.modify_text(out_content)
    with open(out_filename, 'a', encoding='utf-8') as file:
        file.write(response)

if __name__ == '__main__':
    start_time = time.time()
    print('waiting...\n')
    proofreading()
    end_time = time.time()
    print('Time Used: ' + str((end_time - start_time)/60) + 'min')

waiting...

Time Used: 0.1464410940806071min
